# Cryogenic Pump Cleaning Data
### Jared Nielsen


## Out-of-Memory Errors
The `client.csv` files are 1.8GB (14 million rows x 38 cols), 1.2GB, and a few hundred KB, respectively.  
For comparison, `QSS Data Share.csv` is 200MB. Pandas crashes when I load the files, so we might need a better system.  
A 100k-row CSV takes 1.2 seconds, a 1-million row CSV takes 12 seconds to load, and it appears to scale linearly.  
An out-of-memory error is thrown when loading a 14-million row CSV.  

## Chunking & Intermediate Cleaning
So we load the data in 100,000-row chunks, clean and separate the chunks, and append to a CSV file. This reduces RAM usage.  
After saving the intermediate cleaned data, we then load the smaller CSV as a single Pandas dataframe and finish processing it.  
- Merge the `unitsequence` with the `dsname`, or pump type.  
- Separate into cryos and turbos, and keep only the relevant fields.
- Save the intermediate smaller dataframes by appending chunks one at a time.

## Final Cleaning & Saving
- Remove the duplicate `ID` field.
- Upsample from milliseconds up to minutes.
- Remove duplicate minutes on the same pump.
- Save the entire dataframe into the final cleaned data CSV.

In [6]:
from datetime import datetime
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sys

plt.rcParams['figure.figsize'] = 15,3
blue = '#1f77b4'
pd.set_option('precision', 2)

In [7]:
# CSVs to load. For simplicity, we will just work with the first one initially. It's plenty big.
filenames = [
    '20180814_102020_DMOS6_0_client.csv',
    '20180814_102020_DMOS6_0_unit.csv',
    '20181105_130901_DMOS6_0_client.csv',
    '20181105_130901_DMOS6_0_unit.csv',
    '20181105_131740_DMOS6_0_client.csv',
    '20181105_131740_DMOS6_0_unit.csv'
]
folder = '/mnt/pccfs/backed_up/tcs/Archive/'
url_client = folder+filenames[0]
url_unit = folder+filenames[1]

# Declare datatypes (strings/floats) for each field in the CSV.
dtype = {
    "unitsequence": int, # ID, negative is test, positive is production
    "uppervalue": float, # turbo vibrations
    "lowervalue": float,
    "rotorspeed": float, # turbo speed
    "motortemp": float, # turbo
    "controllertemp": float, # turbo
    "dcvoltage": float, # turbo
    "motorcurrent": float, # turbo
    "rotorposition0": str, # turbo, positions of the sensors that determine vibration
    "rotorposition1": str,
    "rotorposition2": str,
    "rotorposition3": str,
    "rotorposition4": str,
    "magcurrent0": float, # turbo balancing factors
    "magcurrent1": float, # 0-7 should all be symmetrical
    "magcurrent2": float,
    "magcurrent3": float,
    "magcurrent4": float,
    "magcurrent5": float,
    "magcurrent6": float,
    "magcurrent7": float,
    "magcurrent8": float, # last two are on top
    "magcurrent9": float,
    "sumcurrents": float, # sum of them
    "tmsactualtemp": float, # turbo heating system temperature. -80 means "NaN"
    "yh": str, # Alcatel turbo, replacement for `rotorposition0`.
    "yb": str,
    "z": float,
    # CRYO fields
    "motorspeed": str, # cryos
    "temp1": float, # cryos, target 65
    "temp2": float, # cryos, target 9/11
    "heater1": float,
    "heater2": float,
    "tcpressure": float, # thermocouple gauge pressure, cryo
    "timestamp": str,
    "currentregen": str, # letter codes
    "alarmstatusbits": str, # 
    "alertstatusbits": str,
    "valvestate": str
}

# Load the key, which maps pump IDs to their type.
x_unit = pd.read_csv(url_unit)

parse_dates = ['timestamp']
start = datetime.now()
x_client_chunks = pd.read_csv(url_client, dtype=dtype, parse_dates=parse_dates, chunksize=100000)
print("done")

done


In [16]:
# TODO: Possibly include rotorspeed in the cryo. It's present in some but not all.
cryo_cols = ['unitsequence', 'timestamp', 'temp1', 'temp2', 
             'heater1', 'heater2', 'tcpressure', 'currentregen']
cryo_vals = ['temp1', 'temp2', 'heater1', 'heater2', 'tcpressure']
# TODO: Possibly include the magcurrents in the turbo.
turbo_cols = ['unitsequence', 'timestamp', 'uppervalue', 'lowervalue', 'rotorspeed', 
              'controllertemp', 'dcvoltage', 'motorcurrent']
turbo_vals = turbo_cols[2:]


def clean_chunk(x_unit, x_client):
    x_client_deduplicate = x_client.drop_duplicates(subset=['unitsequence', 'timestamp'], keep='first')
    x_merge = pd.merge(x_client_deduplicate, x_unit, on='unitsequence')
    x_cryo = x_merge[x_merge['dsname'] == 'CRYO'][cryo_cols]
    x_turbo = x_merge[x_merge['dsname'] == 'SCU'][turbo_cols]
    
    return x_cryo, x_turbo

cryo_dfs = []
turbo_dfs = []
for i, chunk in enumerate(x_client_chunks):
#     print("chunk {}".format(i))
    x_cryo, x_turbo = clean_chunk(x_unit, chunk)
    cryo_dfs.append(x_cryo)
    turbo_dfs.append(x_turbo)
    
# print(len(cryo_dfs))

In [11]:
cryos_csv_duplicates = 'cryos_cleaned_temp.csv'
turbos_csv_duplicates = 'turbos_cleaned_temp.csv'

for i, cryo_df in enumerate(cryo_dfs):
    header = True if i == 0 else False
    cryo_df.to_csv(cryos_csv_duplicates, mode='a', header=header) # takes an exorbitant amount of time
    
for i, turbo_df in enumerate(turbo_dfs):
    header = True if i == 0 else False
    turbo_df.to_csv(turbos_csv_duplicates, mode='a', header=header)

In [13]:
def clean(df):
    df = df.drop("Unnamed: 0", axis='columns',) # remove previous 'ID' field
    df.loc[:,'timestamp'] = df.loc[:,'timestamp'].astype('datetime64[m]') # upsample to nearest minute
    df = df.drop_duplicates(subset=['unitsequence', 'timestamp'], keep='first') # deduplicate
    return df

cryo_csv = 'cryos_cleaned.csv'
turbo_csv = 'turbos_cleaned.csv'

start = datetime.now()
x_cryo = clean(pd.read_csv(cryos_csv_duplicates))
elapsed = (datetime.now() - start).total_seconds()

print("Cryos Shape: {}".format(x_cryo.shape))
print("Time: {}".format(elapsed))

start = datetime.now()
x_turbo = clean(pd.read_csv(turbos_csv_duplicates))
elapsed = (datetime.now() - start).total_seconds()

print("Turbos Shape: {}".format(x_turbo.shape))
print("Time: {}".format(elapsed))

Cryos Shape: (2005273, 8)
Time: 9.792932
Turbos Shape: (409653, 8)
Time: 9.774025


In [15]:
x_cryo.to_csv(cryo_csv, header=True, index=False)
x_turbo.to_csv(turbo_csv, header=True, index=False)
print("Data is Clean!")
print("Stored in {} and {}".format(cryo_csv, turbo_csv))

Data is Clean!
Stored in cryos_cleaned.csv and turbos_cleaned.csv
